# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Import API key
import warnings
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/hansgrice/Desktop/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vila franca do campo,37.7167,-25.4333,64.89,99,100,7.20,PT,1713806860
1,1,adamstown,-25.0660,-130.1015,72.66,74,78,26.98,PN,1713806860
2,2,isafjordur,66.0755,-23.1240,42.84,75,0,3.44,IS,1713806860
3,3,ust-nera,64.5667,143.2000,20.57,100,98,3.04,RU,1713806860
4,4,hamilton,39.1834,-84.5333,62.74,34,0,4.61,US,1713806412


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# coordinates
coordinates = {
    "latitude": [37.7167, -25.0660, 66.0755, 64.5667, 39.1834],
    "longitude": [-25.4333, -130.1015, -23.1240, 143.2000, -84.5333],
    "City": ["vila franca do campo", "adamstown", "isafjordur", "ust-nera", "hamilton"],
    "Humidity": [99, 74, 75, 100, 34]
}

In [4]:
# df coordinates
coordinates_df = pd.DataFrame(coordinates)
coordinates_df



,latitude,longitude,City,Humidity
0,37.7167,-25.4333,vila franca do campo,99
1,-25.0660,-130.1015,adamstown,74
2,66.0755,-23.1240,isafjordur,75
3,64.5667,143.2000,ust-nera,100
4,39.1834,-84.5333,hamilton,34


In [16]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles='OSM',
    hover_cols=['City']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# narrow down cities that fit criteria 
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 28) & (city_data_df["Humidity"] < 60)]

ideal_cities = ideal_cities.dropna()

ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
282,282,sept-iles,50.2001,-66.3821,25.05,49,75,21.85,CA,1713806945
371,371,pont rouge,48.8833,-72.0825,26.89,23,90,13.40,CA,1713806965
490,490,rimouski,48.4488,-68.5240,26.28,39,100,21.07,CA,1713806762


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vila franca do campo,PT,37.7167,-25.4333,99,
1,adamstown,PN,-25.0660,-130.1015,74,
2,isafjordur,IS,66.0755,-23.1240,75,
3,ust-nera,RU,64.5667,143.2000,100,
4,hamilton,US,39.1834,-84.5333,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accommodation.hotel",
    "filter": f"circle:{hotel_df['Lng']},{hotel_df['Lat']},{radius}",
    "bias": f"proximity:{hotel_df['Lng']},{hotel_df['Lat']}",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
adamstown - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
ust-nera - nearest hotel: гостиница "Солнечная"
hamilton - nearest hotel: North Vista Manor
iqaluit - nearest hotel: Frobisher Inn
north shore - nearest hotel: Takapuna International Motor Lodge
port-aux-francais - nearest hotel: Keravel
albany - nearest hotel: Tru
sittwe - nearest hotel: Yuzana Aung Motel 1
bratsk - nearest hotel: Три кита
orumiyeh - nearest hotel: هتل لاله
kununurra - nearest hotel: No hotel found
jose bonifacio - nearest hotel: Assi Palace Hotel
thompson - nearest hotel: Thompson Inn
callao - nearest hotel: Casa Ronald
waitangi - nearest hotel: Hotel Chathams
whitehorse - nearest hotel: Town & Mountain Hotel
yellowknife - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
puerto baquerizo moreno - nearest hotel: Dory's House
balabac - nearest hotel: Kamp Malaya
portl

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vila franca do campo,PT,37.7167,-25.4333,99,Vinha d'Areia Beach Hotel
1,adamstown,PN,-25.0660,-130.1015,74,No hotel found
2,isafjordur,IS,66.0755,-23.1240,75,Hótel Horn
3,ust-nera,RU,64.5667,143.2000,100,"гостиница ""Солнечная"""
4,hamilton,US,39.1834,-84.5333,34,North Vista Manor
...,...,...,...,...,...,...
552,minas de marcona,PE,-15.2119,-75.1103,63,No hotel found
553,praya,CV,14.9215,-23.5087,50,Casa Sodadi
554,oranjemund,NaN,-28.5500,16.4333,95,Op My Stoep
555,curitibanos,BR,-27.2828,-50.5844,32,Hotel Modelo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles='OSM',
    text="Hotel Name",
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)